In [1]:
import os
import warnings
import pandas as pd
pd.set_option('max_colwidth', 400)

## Import data

In [2]:
base_folder = "resources"
save_model_folder = os.path.join(base_folder, "models")
train_dataset_path = os.path.join(base_folder, "train.csv")
test_dataset_path = os.path.join(base_folder, "test.csv")

In [3]:
df = pd.read_csv(train_dataset_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           300000 non-null  int64 
 1   raw_address  300000 non-null  object
 2   POI/street   300000 non-null  object
dtypes: int64(1), object(2)
memory usage: 6.9+ MB


## Preprocess data

In [4]:
POI_LABEL = 'POI'
STREET_LABEL = 'STREET'

def has_overlap(x_start, x_end, y_start, y_end):
    if (x_start == x_end):
        return False
    x = range(x_start, x_end)
    y = range(y_start, y_end)
    if range(max(x[0], y[0]), min(x[-1], y[-1])+1):
        return True
    else:
        return False

def prepare_row(sentence: str, poi_street: str, row: int):
    entities = []
    poi, street = poi_street.split('/')
    street_start_idx = -2
    street_end_idx = -1
    
    if street:
        street_start_idx = sentence.find(street)
        if (street_start_idx != -1):
            street_end_idx = street_start_idx + len(street)
            entities.append((street_start_idx, street_end_idx, STREET_LABEL))
    
    if poi:
        poi_start_idx = sentence.rfind(poi)
        # Only add POI if we can find it in sentence and it doesn't overlap with street
        if (poi_start_idx != -1):
            poi_end_idx = poi_start_idx + len(poi)
            if (not has_overlap(street_start_idx, street_end_idx, poi_start_idx, poi_end_idx)):
                entities.append((poi_start_idx, poi_end_idx, POI_LABEL))
    
    return (sentence, {"entities": entities})

In [5]:
def iterrows_impl(df):
    return pd.Series(prepare_row(row["raw_address"], row["POI/street"], idx) for idx, row in df.iterrows())

In [25]:
TRAIN_DATA = iterrows_impl(df).tolist()

In [71]:
subset = df[df["id"]< 1500]
TRAIN_DATA2 = iterrows_impl(subset).tolist()

## Train using Spacy

In [197]:
import spacy
import random
from spacy.util import minibatch, compounding
from spacy.training import Example
random.seed(42)

In [190]:
# Add our new labels to the pre-trained model
# spacy.require_gpu()
nlp = spacy.load("en_core_web_md") 
ner = nlp.get_pipe('ner')
optimizer = nlp.resume_training()

In [213]:
nlp=spacy.blank("en")
nlp.add_pipe('ner')
ner = nlp.get_pipe('ner')
optimizer = nlp.begin_training()

In [214]:
ner.add_label(POI_LABEL)
ner.add_label(STREET_LABEL)

move_names = list(ner.move_names)

# Pipes we want to train
pipe_exceptions =  ["ner", "trf_tok2vec"] #

# Pipes that should remain unaffected in training
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [215]:
with nlp.disable_pipes(*unaffected_pipes), warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning, module='spacy') 
    examples = []
    
    for text, annots in TRAIN_DATA:    
        examples.append(Example.from_dict(nlp.make_doc(text), annots))
    
#     nlp.initialize(lambda: examples) #enable if using simpler model
    print("Done preprocessing...")
    
  # Training for 30 iterations
    for i in range(30):

    # shuffling examplesbefore every iteration
        random.shuffle(examples)
        losses = {}
    
    # batch up the examples using spaCy's minibatch
        batches = minibatch(examples, size=compounding(16.0, 512.0, 1.01))
        for batch in batches:
            nlp.update(batch, drop=0.2, sgd=optimizer, losses=losses)
        print("Iteration: ", i, ". Losses :", losses['ner'])

Done preprocessing...
Iteration:  0 . Losses : 209617.06030391483
Iteration:  1 . Losses : 146922.1548112241
Iteration:  2 . Losses : 129276.63203384512
Iteration:  3 . Losses : 117265.23319832934
Iteration:  4 . Losses : 109990.9766971165
Iteration:  5 . Losses : 104395.04767885193
Iteration:  6 . Losses : 100317.46871261609
Iteration:  7 . Losses : 96024.44897348138
Iteration:  8 . Losses : 91927.33636539907
Iteration:  9 . Losses : 90913.83107219198
Iteration:  10 . Losses : 87210.1266651245
Iteration:  11 . Losses : 85210.89324504482
Iteration:  12 . Losses : 83439.62068157703
Iteration:  13 . Losses : 81858.33372219517
Iteration:  14 . Losses : 79967.95025527183
Iteration:  15 . Losses : 78469.00969214742
Iteration:  16 . Losses : 76780.77068726646
Iteration:  17 . Losses : 75907.66850004694
Iteration:  18 . Losses : 76434.81916782825
Iteration:  19 . Losses : 73684.33179273798
Iteration:  20 . Losses : 73070.48401601658
Iteration:  21 . Losses : 72598.75959675871
Iteration:  22 .

In [216]:
model_name = "all_model_66563"
nlp.meta['name'] = model_name
nlp.to_disk(save_model_folder + "/" + model_name)

## Use model on test data

In [217]:
nlp2 = spacy.load(save_model_folder + '/all_model_66563')

In [218]:
test_df = pd.read_csv(test_dataset_path)

In [219]:
def apply_model(raw_address:str, idx: int):
    if (idx % 100 == 0):
        print(idx)
    
    # Apply model on address
    doc = nlp2(raw_address)
    poi_street = "/"
    
    # Avoid adding multiple streets/poi if model predicts them
    poi_found = False
    street_found = False
    
    for ent in doc.ents:
        if (ent.label_=='POI' and not poi_found):
            poi_street = ent.text + poi_street
            poi_found = True
        elif (ent.label_=='STREET'and not street_found):
            poi_street += ent.text
            street_found = True
        elif (ent.label_ != 'POI' and ent.label_ != 'STREET'):
            print(ent.label)
            print(idx)
    
    return poi_street

In [220]:
def apply_model_series(df):
    return pd.Series(apply_model(row["raw_address"], idx) for idx, row in df.iterrows())

In [221]:
test_df[test_df['id'] == 23]
apply_model('tebet barat tebet barat vi no 12 4 tebet', 1)

'/tebet barat vi'

In [222]:
test_df['poi/street'] = apply_model_series(test_df)
print(test_df.head(10))
test_df.info()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [223]:
test_df.to_csv('output.csv', columns=['id', 'poi/street'], index=False)